In [1]:
try: import fastkaggle
except ModuleNotFoundError:
    !pip install -Uq fastkaggle fastai py7zr scikit-learn

In [2]:
from fastkaggle import *

In [3]:
# for working with paths in Python, I recommend using `pathlib.Path`
from pathlib import Path

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not iskaggle and not cred_path.exists():
    creds = ''
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [4]:
comp = 'kkbox-music-recommendation-challenge'

path = setup_comp(comp, install='fastai')

In [5]:
path

Path('kkbox-music-recommendation-challenge')

In [6]:
from fastai.tabular.all import *
from fastai.collab import *

path.ls()

(#12) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv.7z'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/songs.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv.7z'),Path('kkbox-music-recommendation-challenge/train.csv.7z'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/members.csv.7z'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv')...]

In [7]:
def is_extracted(file):
    return os.path.isfile(os.path.splitext(file)[0])

for file in path.ls(file_exts=".7z"):
    if not is_extracted(file):
        !py7zr x {str(file)} {str(path)}

In [8]:
path.ls(file_exts=".csv")

(#6) [Path('kkbox-music-recommendation-challenge/train.csv'),Path('kkbox-music-recommendation-challenge/test.csv'),Path('kkbox-music-recommendation-challenge/sample_submission.csv'),Path('kkbox-music-recommendation-challenge/song_extra_info.csv'),Path('kkbox-music-recommendation-challenge/songs.csv'),Path('kkbox-music-recommendation-challenge/members.csv')]

In [9]:
songs = pd.read_csv(path/'train.csv', usecols=('msno', 'song_id', 'target'))

songs.head()

,msno,song_id,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,1


In [10]:
dls = CollabDataLoaders.from_df(songs, bs=64)
dls.show_batch()

,msno,song_id,target
0,MLNXCaZgplGixHprEfq6NOK5m1Ll5nvtYspbdjsffJw=,48PnLVsKQXwlRoNr7779QyZxweYBbpX9z3E1YZm2F/A=,0
1,gGHdfA/CZswpvs8oWNzqgGkc5JfYGqut1arwLVnAMyY=,5lFyg36akDT13RmKEdK/9m2sxrfjreYl1MXU68aReTI=,0
2,EU+1DfnHOk3fhpi3nDt6FhmItjx5k7FY5KYkWlsJOlg=,8Fxu5UXZarINZLkqiJDTl9j299P20budqtNhaLU4Vvc=,1
3,1d4acB2bhEZCFjSRwvO4ls8PrBtvNTlkcAcxYx8FcWE=,gIhnkJdu7/eiNhg4Qfc64iOxYG7C8OwWjrVfwz5RmFA=,0
4,q+58LOnd51IOM4YLXYsenOTH6+33/Mtoq0fpJ1iuvKY=,HfvjLNGhgK6qQVye8aNByn/iJkjv5YexNLFY9AY5TMQ=,1
5,EV+haqw1v8W3llQyva2hU+zvf+hCIKNDfTuoTeyUkmE=,DH9SXL888u6VtOCkF/nqFbmtHlEkYgh9atVYPHfSfo4=,0
6,sdd3ZtNzgtsDR9tdKNEyA9RAdFiAP7OZcEOwnjiBrHQ=,Kqd+taz4vX4q/eP4ZQavikNg6lC8ccbX8aar5PRsMDw=,0
7,L1/OIvtk/e3bFs7ac/FhGFifIAAPBKzdov0kJGq63h4=,UJxg5LL4d4ZVSjGHdjHpCnVpnsjIW4WOTpte/CShRYs=,0
8,Sdu8k2QFzndnEmD/USHiU7G/3M0t1KH5DXgE27E6zCk=,RfC/qUw0sMMB6XLwgVM4G2/L4e8W7CnixpbpDXS8wqg=,0
9,hBIVwkOusT7Bn5/2jYCD0aqzsU3OAvFrWWh1TBuV8Fw=,3Zwlnes20/zfnSgPDAS30cUvdJmKa4+rQ1UEIXrKr3M=,1


In [11]:
from sklearn.metrics import roc_auc_score

def roc_metric(pred, target):
    return roc_auc_score(to_np(target.squeeze()), to_np(pred))

learn = collab_learner(dls, y_range=(-0.1, 1.1), metrics=roc_metric)

In [12]:
collab_learner??

Signature:
collab_learner(
    dls,
    n_factors=50,
    use_nn=False,
    emb_szs=None,
    layers=None,
    config=None,
    y_range=None,
    loss_func=None,
    *,
    opt_func: 'Optimizer | OptimWrapper' = <function Adam at 0x7fea5616d080>,
    lr: 'float | slice' = 0.001,
    splitter: 'callable' = <function trainable_params at 0x7fea5a0b6840>,
    cbs: 'Callback | MutableSequence | None' = None,
    metrics: 'callable | MutableSequence | None' = None,
    path: 'str | Path | None' = None,
    model_dir: 'str | Path' = 'models',
    wd: 'float | int | None' = None,
    wd_bn_bias: 'bool' = False,
    train_bn: 'bool' = True,
    moms: 'tuple' = (0.95, 0.85, 0.95),
    default_cbs: 'bool' = True,
)
Source:   
@delegates(Learner.__init__)
def collab_learner(dls, n_factors=50, use_nn=False, emb_szs=None, layers=None, config=None, y_range=None, loss_func=None, **kwargs):
    "Create a Learner for collaborative filtering on `dls`."
    emb_szs = get_emb_sz(dls, ifnone(emb_szs, {}))
 

In [ ]:
learn.fit_one_cycle(12, 1e-1, wd=0.1)

In [ ]:
learn.model

In [ ]:
torch.save(learn.model.state_dict(), '01.pth')
# learn.model.load_state_dict(torch.load('01.pth'))

In [ ]:
test_df = pd.read_csv(path/'test.csv', usecols=('msno', 'song_id', 'id'))

test_df.head()

In [ ]:
test_dl = learn.dls.test_dl(test_df)
output = learn.get_preds(dl=test_dl, with_decoded=True)
output

In [ ]:
probs,_,indexes = output

In [ ]:
sample_submission = pd.read_csv(path/'sample_submission.csv')
sample_submission

In [ ]:
probs = torch.clip(probs, 0, 1)
test_df['target'] = probs

sub_df = test_df[['id','target']]
sub_df.to_csv(f'submission.csv', index=False)

In [ ]:
!head submission.csv

In [ ]:
if not iskaggle:
    from kaggle import api
    api.competition_submit_cli('submission.csv', 'kkbox-recommendation-3', comp)

In [5]:
if not iskaggle:
    push_notebook('alexchalk', 'poisonous-mushrooms-submission-05',
                  title='Poisonous Mushrooms Submission 05',
                  file='02.ipynb',
                  competition=comp, private=True, gpu=False)

Kernel version 1 successfully pushed.  Please check progress at https://www.kaggle.com/code/alexchalk/poisonous-mushrooms-submission-05
